In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import time
import bs4
import pandas as pd
import re

In [9]:
#webdriver option
opt= webdriver.ChromeOptions()
opt.add_argument('--no-sandbox')
opt.add_argument('--headless')
opt.add_argument('--disable-notifications')
opt.add_argument('--disable-infobars')

#basic code
driver = webdriver.Chrome(executable_path='D:\Github\coba\chromedriver.exe',options=opt)
driver.get('https://shopee.co.id/')
time.sleep(1)

#search
search = driver.find_element(By.XPATH,'//*[@id="main"]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
search.send_keys('skincare')
search.send_keys(Keys.ENTER)
time.sleep(5)

#terlaris
# terlaris = driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[1]/div[1]/div[3]')
# terlaris.click()
# time.sleep(1)

# zoom out
driver.execute_script("document.body.style.zoom='10%'")
time.sleep(2)

# variabel kosong untuk menampung data html
data=str()

# paginasi
for k in range (5) :
    #ambl semua data di page tsb
    data += driver.page_source
    time.sleep(5)
    # navigasi ke page selanjutnya
    next = driver.find_element(By.CSS_SELECTOR,' button.shopee-icon-button.shopee-icon-button--right ')
    driver.execute_script("arguments[0].click();", next)
    time.sleep(5)        
    
driver.close()


C:\Users\MSi\AppData\Local\Temp\ipykernel_31736\1092188061.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='D:\Github\coba\chromedriver.exe',options=opt)


In [10]:
# parse semua data masing masing produk
soup = bs4.BeautifulSoup(data)
all_product = soup.find_all('div',{'class':"col-xs-2-4 shopee-search-item-result__item"})

# variabel kosong untuk menyatukan data
data_dict_list = []

#menata dan merapikan data ke masing masing index
for product in all_product:
    title_element = product.find('div',{'class':'ie3A+n bM+7UW Cve6sh'})
    title_text = title_element.text

    price_element = product.find('div',{'class':'hpDKMN'})
    price_text = price_element.text

    #gunakan if-else jika terdapat data yang kosong
    pricedc_element = product.find('span',{'class':'ZEgDH9'})
    if pricedc_element is None:
        pricedc_text = None
    else:
        pricedc_text = pricedc_element.text
    
    terjual_element = product.find('div',{'class':'r6HknA uEPGHT'})
    if terjual_element is None:
        terjual_text = None
    else:
        terjual_text = terjual_element.text
    
    product_link_element = product.find('a')
    product_link = product_link_element.get('href')
    
    #gabung data dalam dict()
    data_dict = dict()
    data_dict['title'] = title_text
    data_dict['price'] = price_text
    # data_dict['pricedc'] = pricedc_text
    data_dict['sales'] = terjual_text
    data_dict['link'] = product_link
    data_dict_list.append(data_dict)

#ubah ke dalam dataframe menggunkan pandas    
data_df = pd.DataFrame(data_dict_list)
data_df.head(20)

,title,price,sales,link
0,PAKET ACNE CLEAR CREAM JERAWAT PARAH MERADANG ...,Rp145.378Rp106.125,10RB+ Terjual,/PAKET-ACNE-CLEAR-CREAM-JERAWAT-PARAH-MERADANG...
1,Skincare Louvrea paket diamon I Sunscren/L'ouv...,Rp485.000,None,/Skincare-Louvrea-paket-diamon-I-Sunscren-L'ou...
2,JGLOW Best Seller Skincare Paket Super Whiteni...,Rp270.000 - Rp290.000,703 Terjual,/JGLOW-Best-Seller-Skincare-Paket-Super-Whiten...
3,NEW!! FW WHITENING PREMIUM SABUN FACIAL WASH B...,Rp42.378Rp36.022,10RB+ Terjual,/NEW!!-FW-WHITENING-PREMIUM-SABUN-FACIAL-WASH-...
4,YUR SKINCARE PAKET BRIGHTENING WHITENING,Rp260.000Rp228.000,51 Terjual,/YUR-SKINCARE-PAKET-BRIGHTENING-WHITENING-i.74...
5,OMG Oh My Glow Paket Lengkap Skincare Peach Gl...,Rp6.499 - Rp34.899,10RB+ Terjual,/OMG-Oh-My-Glow-Paket-Lengkap-Skincare-Peach-G...
6,PAKET SKINCARE YEPPU YEPPU BY KIYOWO TRAVEL SI...,Rp99.000Rp85.000,"4,5RB Terjual",/PAKET-SKINCARE-YEPPU-YEPPU-BY-KIYOWO-TRAVEL-S...
7,Paket Skincare Pria Brightening 2in1 Sabun Ser...,Rp8.500 - Rp35.000,"1,4RB Terjual",/Paket-Skincare-Pria-Brightening-2in1-Sabun-Se...
8,Paket Leika skincare Penghilang Flek Hitam Di ...,Rp200.000,None,/Paket-Leika-skincare-Penghilang-Flek-Hitam-Di...
9,CREAM PEMUTIH WAJAH [PAKET PENGELUPASAN & PAKE...,Rp125.000,10RB+ Terjual,/CREAM-PEMUTIH-WAJAH-PAKET-PENGELUPASAN-PAKET-...


In [11]:
data_df.to_csv('skincare.csv',index=False,sep=';')

In [151]:
price_1 = 'Rp183.678Rp128.574'
price_2 = 'Rp45.000 - Rp265.000'
price_3 = 'Rp1.524.000'

#bersihkan harga
def clean_price(price):
    result = re.findall(r'[0-9.,]+',price)
    if '-' in price:
        original_price = None
        min_price = result[0]
        max_price = result[1]
    elif len(result) == 2:
        original_price = result[0]
        min_price = result[1]
        max_price = result[1]
    elif len(result) == 1:
        original_price = result[0]
        min_price = result[0]
        max_price = result[0]
        
    price_list =  [original_price,min_price,max_price]
    price_clean_list = []
    for price in price_list:
        if price is None:
            price_clean_list.append(None)
        else:
            price_clean_list.append(int(price.replace('.','')))
    
    return price_clean_list

price_clean_ser = data_df['price'].apply(clean_price)
price_clean_df = pd.DataFrame(price_clean_ser)
price_clean_list = list(price_clean_ser)
price_clean_df = pd.DataFrame(price_clean_list, columns=['Original Price', 'Min Price', 'Max Price'])
price_data_df = pd.concat([data_df,price_clean_df],axis=1)

#drop price dan eksport csv
new = price_data_df.drop(['price'],axis=1)
new.to_csv('pigura600_1.csv',index=False,sep=';')


In [98]:
def penjualan_clean(jual):
    result = re.findall(r'([0-9,]+RB\+|[0-9,]+RB|[0-9,]+)',jual)
    jual_list= []
    for jual in result:
        if jual is None:
            jual_list.append(None)
        else:
            jual_list.append(str(jual))
                

    return jual_list
    
data_df['terjual'] = data_df['terjual'].astype(str)
df = data_df['terjual'].apply(penjualan_clean)
df1 = df.replace(',','.')
df2 = df1.replace({'^(BR)': '*1e3', '(RB+)': '*1e3'}, regex=True).map(pd.eval).astype(int)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 1)

Bagian bawah bisa digunakan namun masih ada kendala terkait kerapian dari data yang di dapatkan dalam file csvnya

In [70]:
with open('shopee_skincare_paganation2.csv','w') as file:
    file.write("nama; harga; harga_diskon; terjual \n")

    #webdriver option
    opt= webdriver.ChromeOptions()
    # opt.add_argument('--no-sandbox')
    opt.add_argument('--headless')
    # opt.add_argument('--disable-notifications')
    # opt.add_argument('--disable-infobars')

    driver = webdriver.Chrome(executable_path='D:\Github\coba\chromedriver.exe',options=opt)
    driver.get('https://shopee.co.id/')

    time.sleep(1)

    #search
    search = driver.find_element(By.XPATH,'//*[@id="main"]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
    search.send_keys('skincare')
    search.send_keys(Keys.ENTER)
    time.sleep(5)

    #terlaris
    terlaris = driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[1]/div[1]/div[3]')
    terlaris.click()
    time.sleep(1)

    driver.execute_script("document.body.style.zoom='10%'")
    time.sleep(2)

    for k in range (3) :
        #ambl semua data di page tsb
        namas= driver.find_elements(By.XPATH,'//div[@class="ie3A+n bM+7UW Cve6sh"]')
        hargas= driver.find_elements(By.XPATH,'//div[@class="hpDKMN"]')
        hargas_diskon= driver.find_elements(By.XPATH,'//span[@class="ZEgDH9"]')
        terjuals = driver.find_elements(By.XPATH,'//div[@class="r6HknA uEPGHT"]')

        with open('shopee_skincare_paganation2.csv','a', encoding="utf-8") as file:
            for i in range (len(namas)-3) :
                file.write(namas[i].text + ";" + hargas[i].text + ";" + hargas_diskon[i].text + ";" + terjuals[i].text + "\n")

            next = driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[3]/div/button[8]')
            driver.execute_script("arguments[0].click();", next)
            time.sleep(15)        
        file.close()
# driver.close()

C:\Users\MSi\AppData\Local\Temp\ipykernel_21748\3033251909.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='D:\Github\coba\chromedriver.exe',options=opt)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main"]/div/div[2]/div/div/div[2]/div[3]/div[3]/div/button[8]"}
  (Session info: headless chrome=107.0.5304.107)
Stacktrace:
Backtrace:
	Ordinal0 [0x00591ED3+2236115]
	Ordinal0 [0x005292F1+1807089]
	Ordinal0 [0x004366FD+812797]
	Ordinal0 [0x004655DF+1005023]
	Ordinal0 [0x004657CB+1005515]
	Ordinal0 [0x00497632+1209906]
	Ordinal0 [0x00481AD4+1120980]
	Ordinal0 [0x004959E2+1202658]
	Ordinal0 [0x004818A6+1120422]
	Ordinal0 [0x0045A73D+960317]
	Ordinal0 [0x0045B71F+964383]
	GetHandleVerifier [0x0083E7E2+2743074]
	GetHandleVerifier [0x008308D4+2685972]
	GetHandleVerifier [0x00622BAA+532202]
	GetHandleVerifier [0x00621990+527568]
	Ordinal0 [0x0053080C+1837068]
	Ordinal0 [0x00534CD8+1854680]
	Ordinal0 [0x00534DC5+1854917]
	Ordinal0 [0x0053ED64+1895780]
	BaseThreadInitThunk [0x76DBFEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77D57BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77D

In [49]:
import pandas as pd
import csv

In [47]:
nama = []
harga = []
harga_diskon = []
terjual = []

for l in range(len(namas)):
    nama.append(namas[l].text)

for l in range(len(hargas)):
    harga.append(hargas[l].text)

for l in range(len(hargas_diskon)):
    harga_diskon.append(hargas_diskon[l].text)

for l in range(len(terjuals)):
    terjual.append(terjuals[l].text)

listCols = ['nama','harga','harga_diskon','terjual']
data_dict=dict(zip(
    listCols,(
    nama,
    harga,
    harga_diskon,
    terjual
)))    


In [60]:
with open('rapi.csv','w',encoding='utf-8') as file :
    w=csv.DictWriter(file,data_dict.keys())
    w.writeheader()
    w.writerow(data_dict)